
<div id="sec_intro_to_intro" style="direction:ltl;line-height:300%;">
	<font face="XB Zar" size=5>
		<font color='crimson' size=7>
            Hypothesis Testing
        </font>
	</font>
</div>

<div id="sec_intro_to_intro" style="direction:ltl;line-height:300%;">
	<font face="XB Zar" size=5>
         We posit that X denotes the three designated coins volume, while Y signifies all other coins volume.
	</font>
</div>

\begin{align*}
H_0 &: \text{Mean of } \textit{X} \leq \text{Mean of } \textit{Y} \\
H_1 &: \text{Mean of } \textit{X} > \text{Mean of } \textit{Y}
\end{align*}



<div id="sec_intro_to_intro" style="direction:ltl;line-height:300%;">
	<font face="XB Zar" size=5>
		<font color='aqua' size=5>
            Importing the Necessary Libraries
        </font>
	</font>
</div>

In [11]:
import pandas as pd
from scipy import stats
from sqlalchemy import create_engine
import plotly.graph_objects as go
from scipy.stats import gaussian_kde
import numpy as np

<div id="sec_intro_to_intro" style="direction:ltl;line-height:300%;">
	<font face="XB Zar" size=5>
		<font color='aqua' size=5>
            Database connection setup
        </font>
	</font>
</div>

In [12]:
username = "root"
password = "root"
server = "localhost"
database_url = f"mysql+mysqlconnector://{username}:{password}@{server}:3306/quera_project_phase_1?charset=utf8mb4"
engine = create_engine(database_url)

<div id="sec_intro_to_intro" style="direction:ltl;line-height:300%;">
	<font face="XB Zar" size=5>
		<font color='aqua' size=5>
            Which Side is better?
        </font>
	</font>
</div>

In [13]:
btc_eth_usdt_query = """SELECT
                            daily_market.c_id, 
                            daily_market.date, 
                            daily_market.volume 
                        FROM
                            daily_market
                        JOIN 
                            coins ON coins.id = daily_market.c_id
                        WHERE 
                            coins.name IN ('Bitcoin', 'Ethereum', 'Thether USDt')"""


others_query = """SELECT
                    daily_market.c_id,
                    daily_market.date,
                    daily_market.volume 
                  FROM 
                    daily_market
                  JOIN 
                    coins ON coins.id = daily_market.c_id
                  WHERE 
                    coins.name NOT IN ('Bitcoin', 'Ethereum', 'Thether USDt')"""

<div id="sec_intro_to_intro" style="direction:ltl;line-height:300%;">
	<font face="XB Zar" size=5>
		<font color='aqua' size=5>
            Reading Queries
        </font>
	</font>
</div>

In [14]:
btc_eth_usdt_results = pd.read_sql_query(btc_eth_usdt_query, engine)
others_results = pd.read_sql_query(others_query, engine)

<div id="sec_intro_to_intro" style="direction:ltl;line-height:300%;">
	<font face="XB Zar" size=5>
		<font color='aqua' size=5>
            Printing data
        </font>
	</font>
</div>

In [15]:
print(btc_eth_usdt_results)
print(others_results)

     c_id        date        volume
0       1  2023-08-31  2.018100e+10
1       1  2023-08-30  1.634370e+10
2       1  2023-08-29  2.936840e+10
3       1  2023-08-28  1.100280e+10
4       1  2023-08-27  6.913770e+09
..    ...         ...           ...
723     2  2022-09-06  2.294610e+10
724     2  2022-09-05  1.306050e+10
725     2  2022-09-04  8.884150e+09
726     2  2022-09-03  9.516830e+09
727     2  2022-09-02  1.770850e+10

[728 rows x 3 columns]
       c_id        date       volume
0       154  2023-08-31    8818780.0
1       154  2023-08-30    5742330.0
2       154  2023-08-29    8196430.0
3       154  2023-08-28    8435250.0
4       154  2023-08-27    5308190.0
...     ...         ...          ...
69592   105  2022-09-06  118080000.0
69593   105  2022-09-05   72999600.0
69594   105  2022-09-04   52849200.0
69595   105  2022-09-03   46434400.0
69596   105  2022-09-02   86483600.0

[69597 rows x 3 columns]


<div id="sec_intro_to_intro" style="direction:ltl;line-height:300%;">
	<font face="XB Zar" size=5>
		<font color='aqua' size=5>
            Now, we will examine the normality of the distributions:
        </font>
	</font>
</div>

In [16]:
btc_eth_usdt = btc_eth_usdt_results["volume"]


hist = go.Histogram(
    x=btc_eth_usdt_results["volume"], opacity=0.75, name="Histogram")


x_vals = np.linspace(
    min(btc_eth_usdt_results["volume"]), max(
        btc_eth_usdt_results["volume"]), 4000
)
kde = gaussian_kde(btc_eth_usdt_results["volume"])
y_vals = kde(x_vals)

kde_trace = go.Scatter(x=x_vals, y=y_vals, mode="lines",
                       name="KDE", yaxis="y2")

layout_dual_y = go.Layout(
    title="Histogram and KDE for daily volume with Dual Y-Axis for 3 Specific Coins",
    xaxis=dict(title="Value"),
    yaxis=dict(title="Histogram Count"),
    yaxis2=dict(title="KDE Density", overlaying="y", side="right"),
    barmode="overlay",
)

fig_dual_y = go.Figure(data=[hist, kde_trace], layout=layout_dual_y)
fig_dual_y.show()

<font color='#add8e6' size=5></font>

In [17]:
others = others_results["volume"]

hist = go.Histogram(x=others_results["volume"], opacity=0.75, name="Histogram")


x_vals = np.linspace(min(others_results["volume"]), max(
    others_results["volume"]), 4000)
kde = gaussian_kde(others_results["volume"])
y_vals = kde(x_vals)

kde_trace = go.Scatter(x=x_vals, y=y_vals, mode="lines",
                       name="KDE", yaxis="y2")

lower_limit = np.percentile(others_results["volume"], 1)
upper_limit = np.percentile(others_results["volume"], 99)

layout_dual_y = go.Layout(
    title="Histogram and KDE for volume with Dual Y-Axis for other Coins",
    xaxis=dict(title="Value", range=[lower_limit, upper_limit]),
    yaxis=dict(title="Histogram Count"),
    yaxis2=dict(title="KDE Density", overlaying="y", side="right"),
    barmode="overlay",
)

fig_dual_y = go.Figure(data=[hist, kde_trace], layout=layout_dual_y)
fig_dual_y.show()

<div id="sec_intro_to_intro" style="direction:ltl;line-height:300%;">
	<font face="XB Zar" size=5>
		<font color='aqua' size=5>
            Checking the equality of variances:
        </font>
	</font>
</div>

In [18]:
var_btc_eth_usdt = np.var(btc_eth_usdt)
var_others = np.var(others)

print(var_btc_eth_usdt)
var_others

1.5388044807116177e+20


7.692068202958217e+18

<div id="sec_intro_to_intro" style="direction:ltl;line-height:300%;">
	<font face="XB Zar" size=5>
         From the graphs, it's evident that the distributions aren't normal, making the t-test inapplicable. Instead, we'll employ the Mann-Whitney test, a non-parametric method that doesn't necessitate conditions like normality or equality of variance.
	</font>
</div>

<div id="sec_intro_to_intro" style="direction:ltl;line-height:300%;">
	<font face="XB Zar" size=5>
         We posit that X denotes the three designated coins volume, while Y signifies all other coins volume.
	</font>
</div>

\begin{align*}
H_0 &: \text{The distribution of } \textit{X} \text{ is the same as the distribution of } \textit{Y} \\
H_1 &: \text{The distribution of } \textit{X} \text{ is different from the distribution of } \textit{Y} \\
& \\
& \text{In other words,} \\
& \\
H_0 &: \text{Median (or Mean) of } \textit{X} = \text{Median (or Mean) of } \textit{Y} \\
H_1 &: \text{Median (or Mean) of } \textit{X} = \text{Median (or Mean) of } \textit{Y}
\end{align*}



<div id="sec_intro_to_intro" style="direction:ltl;line-height:300%;">
	<font face="XB Zar" size=5>
                The Mann-Whitney U test, is chosen because the data does not follow a normal distribution. This non-parametric test compares the distributions of two independent samples without making strict assumptions about the specific distribution from which the data originated.<br>
                We reference the medians when interpreting the results of the Mann-Whitney U test because the median, being a measure of central tendency, accurately reflects the center of a distribution. It remains unaffected by outliers, making it a more reliable measure especially for non-normally distributed data. Though the test doesn't directly pit medians (or means) against each other, a significant outcome indicates a difference in the central tendencies of the two groups.
        </font>
</div>

<div id="sec_intro_to_intro" style="direction:ltl;line-height:300%;">
	<font face="XB Zar" size=5>
		<font color='aqua' size=5>
            Performing Man-Whitney test:
        </font>
	</font>
</div>

In [19]:
statistic, p_value = stats.mannwhitneyu(btc_eth_usdt, others)
p_value

0.0

<div id="sec_intro_to_intro" style="direction:ltl;line-height:300%;">
	<font face="XB Zar" size=5>
                As evident, the null hypothesis is dismissed due to a p-value of zero, indicating a discrepancy between the means of the two datasets. But which dataset holds the edge? To discern the direction of this variation, we'll compute the means of the datasets:
        </font>
</div>

In [20]:
mean_btc_eth_usdt = np.mean(btc_eth_usdt)
mean_others = np.mean(others)

print("mean of 3 coins:", mean_btc_eth_usdt)
print("mean of other coisn:", mean_others)

mean of 3 coins: 15937977293.956043
mean of other coisn: 296327374.3089903


<div id="sec_intro_to_intro" style="direction:ltl;line-height:300%;">
	<font face="XB Zar" size=5>
                Based on the obtained averages, we can conclude that the null hypothesis is rejected in favor of the alternative. This implies that the average daily trading volumes of 'Bitcoin', 'Ethereum', and 'Tether USDt' are significantly higher than those of other cryptocurrencies
        </font>
</div>